In [1]:
!pip install pytesseract transformers[torch] datasets rouge-score nltk py7zr --upgrade #tensorboard
!pip install accelerate -U
!pip install bert_score
!pip install evaluate

In [2]:
dataset_id = "knkarthick/dialogsum"
from datasets import load_dataset

dataset = load_dataset(dataset_id)

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-large"

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [3]:
dataset["train"][0]["topic"]

'get a check-up'

In [4]:
import json

from datasets.dataset_dict import DatasetDict
ds = DatasetDict()

def update_dataset(example, index):
    example["dialogue"] = f"Summarize the following dialogue towards the topic of {dataset['train'][index]['topic']}.\n" + dataset["train"][index]["dialogue"]
    return example

ds["train"] = dataset["train"].map(update_dataset, with_indices=True)

def update_dataset(example, index):
    example["dialogue"] = f"Summarize the following dialogue towards the topic of {dataset['test'][index]['topic']}.\n" + dataset["test"][index]["dialogue"]
    return example

ds["test"] = dataset["test"].map(update_dataset, with_indices=True)

from datasets import concatenate_datasets

tokenized_inputs = concatenate_datasets([ds["train"], ds["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])

tokenized_targets = concatenate_datasets([ds["train"], ds["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])

def preprocess_function(sample,padding="max_length"):

    # the instructions can be rewritten based on need

    # NOTE I tried the <I></I> and <sep></sep>, not much difference

    # and towards the topic of {sample['topic']}.
    inputs = [f"Summarize the following dialogue with helping information after each utterance (between the special separator) and" + item for item in sample["dialogue"]]

    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# the problem is here
tokenized_dataset = ds.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
# tokenized_dataset["train"] = updated_train.map(preprocess_function, with_indices=True, batched=True,remove_columns=["dialogue", "summary", "id"])
# tokenized_dataset["test"] = updated_test.map(preprocess_function, with_indices=True, batched=True,remove_columns=["dialogue", "summary", "id"])


Map:   0%|          | 0/13960 [00:00<?, ? examples/s]

Map:   0%|          | 0/13960 [00:00<?, ? examples/s]

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [5]:
ds["test"]["dialogue"][0]

"Summarize the following dialogue towards the topic of communication method.\n#Person1#: Ms. Dawson, I need you to take a dictation for me.\n#Person2#: Yes, sir...\n#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?\n#Person2#: Yes, sir. Go ahead.\n#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.\n#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?\n#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.\n#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.\n#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Mes

In [6]:
tokenized_dataset["test"]["input_ids"][0] == tokenized_dataset["test"]["input_ids"][1]

False

In [7]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
# model_id="google/flan-t5-large"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [13]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    fp16=False,
    learning_rate=5e-5,
    num_train_epochs=2,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.931400,1.075916,40.744300,17.563700,35.206700,36.889300,18.883333
2,0.817200,1.065560,41.055000,18.099800,35.630800,37.408200,18.845333


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=6230, training_loss=0.8967120567064607, metrics={'train_runtime': 4948.4152, 'train_samples_per_second': 5.036, 'train_steps_per_second': 1.259, 'total_flos': 5.743481733513216e+16, 'train_loss': 0.8967120567064607, 'epoch': 2.0})

In [15]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 1.065560221672058,
 'eval_rouge1': 41.055,
 'eval_rouge2': 18.0998,
 'eval_rougeL': 35.6308,
 'eval_rougeLsum': 37.4082,
 'eval_gen_len': 18.845333333333333,
 'eval_runtime': 373.4175,
 'eval_samples_per_second': 4.017,
 'eval_steps_per_second': 1.004,
 'epoch': 2.0}

In [ ]:
# {'eval_loss': 1.065560221672058,
#  'eval_rouge1': 41.055,
#  'eval_rouge2': 18.0998,
#  'eval_rougeL': 35.6308,
#  'eval_rougeLsum': 37.4082,
#  'eval_gen_len': 18.845333333333333,
#  'eval_runtime': 373.4175,
#  'eval_samples_per_second': 4.017,
#  'eval_steps_per_second': 1.004,
#  'epoch': 2.0}

In [16]:
def my_tokenize(examples):
    return tokenizer(examples['dialogue'], padding=True, truncation=True, max_length=512)
td = ds["test"].map(my_tokenize, batched=True)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [17]:
ps = trainer.predict(td, max_length = 100)




In [18]:
import numpy as np
predictions = np.where(ps.predictions != -100, ps.predictions, tokenizer.pad_token_id)
predictions = [tokenizer.decode(ids, skip_special_tokens=True, max_length = 100) for ids in predictions]
references = [s["summary"] for s in dataset["test"]]

In [19]:
print(predictions[0])
print(predictions[1])
print(predictions[2])
print(predictions[10])
print(predictions[11])
print(predictions[12])

#Person1# asks Ms. Dawson to take a dictation for him. Ms. Dawson tells #Person1# that all office communications are restricted to email correspondence and official memos and the use of Instant Message programs by employees during working hours is strictly prohibited.
#Person1# asks Ms. Dawson to take a dictation for him. Ms. Dawson tells #Person1# that all office communications are restricted to email correspondence and official memos and the use of Instant Message programs by employees during working hours is strictly prohibited.
#Person1# asks Ms. Dawson to take a dictation for #Person1#. Ms. Dawson tells #Person1# that all office communications are restricted to email correspondence and official memos and the use of Instant Message programs by employees during working hours is strictly prohibited. Ms. Dawson asks #Person1# to get the memo typed up and distributed to all employees before 4 pm.
#Person1# and Brian are at a birthday party. Brian compliments #Person1#'s appearance and 

In [21]:
from rouge_score import rouge_scorer, scoring

def compute_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    aggregator = scoring.BootstrapAggregator()

    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        aggregator.add_scores(scores)

    # 计算最终的平均分数
    result = aggregator.aggregate()

    # 转换为易于阅读的格式
    final_scores = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return final_scores

rouge_scores = compute_rouge(predictions, references)
print(rouge_scores)


{'rouge1': 47.379407412384346, 'rouge2': 21.80985727265991, 'rougeL': 39.1405311997194}


In [22]:
from evaluate import load
bertscore = load("bertscore")

results = bertscore.compute(predictions=predictions, references=references, lang="en")

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# predictions
with open("flan_t5_large_dialogsum_topic.txt", 'w', encoding='utf-8') as f:
    for string in predictions:
        f.write(string + '\n')

from google.colab import files
files.download('flan_t5_large_dialogsum_topic.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>